In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Download necessary NLTK data

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saiun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saiun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load the data

In [3]:
file_path = 'reviews (1).csv'
data = pd.read_csv(file_path)
data.head()# Inspect the data

,reviewId,userName,userImage,content,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId
0,0197c118-5c6f-4a7b-894c-970023d1a350,Mar Zur,https://play-lh.googleusercontent.com/a/ACg8oc...,I have the same recurring tasks to do every da...,11,4.16.6.2,22-07-2020 13:13,Our team will be happy to look into it for you...,23-07-2020 16:32,4.16.6.2,most_relevant,com.anydo
1,94868fb5-a21d-4ef9-ab85-81b2ed3d0785,Devin Rivera,https://play-lh.googleusercontent.com/a-/ALV-U...,"Instead of shopping around, I downloaded Any.d...",8,NaN,08-12-2020 06:24,We are not aware of any issues with randomized...,10-12-2020 09:38,NaN,most_relevant,com.anydo
2,825da34e-f65d-4ef3-991d-02d5291820d6,Heidi Kinsley,https://play-lh.googleusercontent.com/a/ACg8oc...,Why does every once in a while... out of the b...,6,5.11.1.2,09-07-2021 13:51,Sorry to hear that! It sounds like you might h...,11-07-2021 11:16,5.11.1.2,most_relevant,com.anydo
3,a49c2875-651a-4c33-b79c-5813780d659e,Daniel Keller,https://play-lh.googleusercontent.com/a/ACg8oc...,Terrible Update! This app used to be perfect f...,5,NaN,16-11-2020 01:50,Please note that the tasks in your tasks view ...,17-11-2020 09:31,NaN,most_relevant,com.anydo
4,9482c75e-2e63-46ab-8c94-47273dd6a829,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This app is deceivingly terrible. There are so...,20,4.14.0.4,31-01-2019 16:19,"Hi Ryan, it sounds like you are describing our...",05-02-2019 11:52,4.14.0.4,most_relevant,com.anydo


In [4]:
# Print column names to identify the correct label column
print(data.columns)

Index(['reviewId', 'userName', 'userImage', 'content', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion',
       'sortOrder', 'appId'],
      dtype='object')


# Text preprocessing

In [5]:
# Function for text preprocessing
def preprocess_text(text):
    # Lower case
    text = text.lower()
    
    # Remove links
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove new lines
    text = text.replace('\n', ' ')
    
    # Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text
data.head()

,reviewId,userName,userImage,content,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId
0,0197c118-5c6f-4a7b-894c-970023d1a350,Mar Zur,https://play-lh.googleusercontent.com/a/ACg8oc...,I have the same recurring tasks to do every da...,11,4.16.6.2,22-07-2020 13:13,Our team will be happy to look into it for you...,23-07-2020 16:32,4.16.6.2,most_relevant,com.anydo
1,94868fb5-a21d-4ef9-ab85-81b2ed3d0785,Devin Rivera,https://play-lh.googleusercontent.com/a-/ALV-U...,"Instead of shopping around, I downloaded Any.d...",8,NaN,08-12-2020 06:24,We are not aware of any issues with randomized...,10-12-2020 09:38,NaN,most_relevant,com.anydo
2,825da34e-f65d-4ef3-991d-02d5291820d6,Heidi Kinsley,https://play-lh.googleusercontent.com/a/ACg8oc...,Why does every once in a while... out of the b...,6,5.11.1.2,09-07-2021 13:51,Sorry to hear that! It sounds like you might h...,11-07-2021 11:16,5.11.1.2,most_relevant,com.anydo
3,a49c2875-651a-4c33-b79c-5813780d659e,Daniel Keller,https://play-lh.googleusercontent.com/a/ACg8oc...,Terrible Update! This app used to be perfect f...,5,NaN,16-11-2020 01:50,Please note that the tasks in your tasks view ...,17-11-2020 09:31,NaN,most_relevant,com.anydo
4,9482c75e-2e63-46ab-8c94-47273dd6a829,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This app is deceivingly terrible. There are so...,20,4.14.0.4,31-01-2019 16:19,"Hi Ryan, it sounds like you are describing our...",05-02-2019 11:52,4.14.0.4,most_relevant,com.anydo


# Apply preprocessing to the text data

In [6]:
# Replace NaN values in the 'content' column with empty strings
data['content'] = data['content'].fillna("")

# Apply preprocessing to the text data
data['cleaned_text'] = data['content'].apply(preprocess_text)

# Display the original content and cleaned text for the first few rows
data[['content', 'cleaned_text']].head()

,content,cleaned_text
0,I have the same recurring tasks to do every da...,recur task everi day need todolist remind buzz...
1,"Instead of shopping around, I downloaded Any.d...",instead shop around download anydo wide use se...
2,Why does every once in a while... out of the b...,everi blue app ask updat acct email everyth ta...
3,Terrible Update! This app used to be perfect f...,terribl updat app use perfect plan certain tas...
4,This app is deceivingly terrible. There are so...,app deceivingli terribl realli nice design fea...


# Vectorize the text data

In [7]:
# Handle missing values in 'content' column
# Drop rows with missing 'content' values
data = data.dropna(subset=['content'])
# For example, we classify a review as positive (1) if it contains the word 'good', otherwise negative (0)
data['label'] = data['content'].apply(lambda x: 1 if 'good' in x.lower() else 0)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['content'])

# Use the created 'label' column as labels
y = data['label']

# Verify the transformation
print(X.shape)
print(y.value_counts())

(16787, 14323)
label
0    14462
1     2325
Name: count, dtype: int64


# Split the dataset

In [8]:
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize models

In [9]:
models = {
    "Naive Bayes": MultinomialNB()
}

# Train and evaluate models

In [10]:
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Validate the model
    y_val_pred = model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"{model_name} Validation Accuracy: {val_accuracy * 100:.2f}%")
    
    # Test the model
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"{model_name} Test Accuracy: {test_accuracy * 100:.2f}%")

Naive Bayes Validation Accuracy: 87.13%
Naive Bayes Test Accuracy: 87.61%


# Binary Search for Models

In [11]:
# Perform binary search for Naive Bayes
param_ranges = {'alpha': [0.1, 10]}  # Initial range for alpha parameter

best_score = 0
best_params_binary = {}

for param, (min_val, max_val) in param_ranges.items():
    while min_val < max_val:
        mid_val = (min_val + max_val) / 2
        model = MultinomialNB(alpha=mid_val)
        model.fit(X_train, y_train)
        score = model.score(X_train, y_train)
        if score > best_score:
            best_score = score
            best_params_binary[param] = mid_val
        if score > best_score:
            min_val = mid_val + 1
        else:
            max_val = mid_val

# Train Naive Bayes with the best parameters found by binary search
best_model_binary = MultinomialNB(alpha=best_params_binary['alpha'])
best_model_binary.fit(X_train, y_train)

y_pred_train_binary = best_model_binary.predict(X_train)
y_pred_test_binary = best_model_binary.predict(X_test)

print("Best Parameters:", best_params_binary)
print("\nTraining Data Classification Report:")
print(classification_report(y_train, y_pred_train_binary))
print("\nTest Data Classification Report:")
print(classification_report(y_test, y_pred_test_binary))

Best Parameters: {'alpha': 0.10120849609375}

Training Data Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     10093
           1       0.99      0.53      0.69      1657

    accuracy                           0.93     11750
   macro avg       0.96      0.77      0.83     11750
weighted avg       0.94      0.93      0.92     11750


Test Data Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      2188
           1       0.80      0.36      0.49       331

    accuracy                           0.90      2519
   macro avg       0.85      0.67      0.72      2519
weighted avg       0.90      0.90      0.89      2519

